In [19]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [20]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [21]:
df = pd.read_csv('countries.csv', encoding = "utf-16")

In [22]:
df[df['city'] == 'London']['lat']

0    51.507322
Name: lat, dtype: float64

In [30]:
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
for c in df['city']:
    params = {
		"latitude": df[df['city'] == c]['lat'],
		"longitude": df[df['city'] == c]['lon'],
		"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone"],
		"timezone": "auto",
		"start_date": "2020-01-01",
		"end_date": "2024-10-22"}
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]


OpenMeteoRequestsError: {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}

Coordinates 36.14999771118164°N -5.350000381469727°E
Elevation 13.0 m asl
Timezone b'Europe/Gibraltar' b'CEST'
Timezone difference to GMT+0 7200 s


In [25]:
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ozone = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                           date       pm10  pm2_5  carbon_monoxide  \
0     2019-12-31 22:00:00+00:00        NaN    NaN              NaN   
1     2019-12-31 23:00:00+00:00        NaN    NaN              NaN   
2     2020-01-01 00:00:00+00:00  20.400000    5.4            130.0   
3     2020-01-01 01:00:00+00:00  24.100000    7.1            130.0   
4     2020-01-01 02:00:00+00:00  22.100000    6.5            129.0   
...                         ...        ...    ...              ...   
42163 2024-10-22 17:00:00+00:00  14.100000    9.0            129.0   
42164 2024-10-22 18:00:00+00:00  14.800000    9.1            130.0   
42165 2024-10-22 19:00:00+00:00  15.600000    9.1            132.0   
42166 2024-10-22 20:00:00+00:00  17.100000    9.2            133.0   
42167 2024-10-22 21:00:00+00:00  17.200001   10.0            135.0   

       nitrogen_dioxide  sulphur_dioxide  ozone  
0                   NaN              NaN    NaN  
1                   NaN              NaN    NaN  
2        

In [26]:
hourly_dataframe.to_csv('air_quality_data.csv', index=False)

In [27]:
data = pd.read_csv('air_quality_data.csv')

In [28]:
data

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone
0,2019-12-31 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-31 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 00:00:00+00:00,20.4,5.4,130.0,0.1,0.0,84.0
3,2020-01-01 01:00:00+00:00,24.1,7.1,130.0,0.1,0.0,83.0
4,2020-01-01 02:00:00+00:00,22.1,6.5,129.0,0.1,0.0,83.0
...,...,...,...,...,...,...,...
42163,2024-10-22 17:00:00+00:00,14.1,9.0,129.0,15.3,5.6,73.0
42164,2024-10-22 18:00:00+00:00,14.8,9.1,130.0,16.0,5.5,70.0
42165,2024-10-22 19:00:00+00:00,15.6,9.1,132.0,16.1,5.3,72.0
42166,2024-10-22 20:00:00+00:00,17.1,9.2,133.0,15.0,5.3,72.0
